In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel

from utils import read_csv_mp, oversample, CorrCoefChunk
from config import X_TRAIN_MP, Y_TRAIN_PATH

# Correlation coefficient matrix calculation (oversampled data)

In [2]:
random_state = 0
X = read_csv_mp(X_TRAIN_MP, asarray=True)
y = pd.read_csv(Y_TRAIN_PATH, header=None)
#X_over, _ = oversample(X, y, random_state=random_state)

In [ ]:
X.shape

In [ ]:
X_over.iloc[:,-1]

## Single matrix

In [16]:
def corrcoef(m):
    A = np.asarray(m)
    A -= np.average(A, axis=0)[None,:]   
    c = np.dot(A.T, A)        
    stdev = np.sqrt(np.diag(c))
    corr = c / stdev[:,None] / stdev[None,:]
    return corr

def corrcoef_ein(m):
    A = np.asarray(m)
    A -= np.einsum('np->p', A, optimize=True) / np.float32(A.shape[0])
    c = np.einsum('nq,np->qp', A, A, optimize=True)
    stdev = np.sqrt(np.einsum('pp->p', c, optimize=True))
    corr = c / stdev[:,None] / stdev[None,:]
    return corr

In [77]:
corr = corrcoef(X_over.iloc[:,:10000])
corr_ein = corrcoef_ein(X_over.iloc[:,:10000])
corr_np = np.corrcoef(X_over.iloc[:,:10000], rowvar=False)

np.allclose(corr_np, corr, atol=0.000001) and np.allclose(corr_np, corr_ein, atol=0.000001)

True

#### corrcoef()

In [18]:
%timeit corrcoef(X[:,:10000])

1.08 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
%timeit corrcoef(X_over.iloc[:,:20000])

3.77 s ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### corrcoef_ein()

In [81]:
%timeit corrcoef_ein(X_over.iloc[:,:10000])

1.32 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%timeit corrcoef_ein(X_over.iloc[:,:20000])

3.77 s ± 8.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### np.corrcoef()

In [83]:
%timeit np.corrcoef(X_over.iloc[:,:10000], rowvar=False)

1.93 s ± 3.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%timeit np.corrcoef(X_over.iloc[:,:20000], rowvar=False)

7.6 s ± 203 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Two matrices

In [85]:
def two_mat_corrcoef(m1, m2):
    A = np.asarray(m1)
    B = np.asarray(m2)
    
    A -= np.average(A, axis=0)
    B -= np.average(B, axis=0)
    
    c = np.dot(B.T, A)
    
    ssq_A = np.sum(np.square(A), axis=0)
    ssq_B = np.sum(np.square(B), axis=0)
    norm = np.sqrt(np.outer(ssq_B, ssq_A))    
    return c / norm

def two_mat_corrcoef_ein(m1, m2, float32=False):
    A = np.asarray(m1)
    B = np.asarray(m2)
    
    A -= np.einsum('np->p', A, optimize=True) / np.float32(A.shape[0])
    B -= np.einsum('nq->q', B, optimize=True) / np.float32(B.shape[0]) # A.shape[0] should be equal to B.shape[0]
    c = np.einsum('nq,np->qp', B, A, optimize=True)
    
    ssq_A = np.einsum('np,np->p', A, A, optimize=True)
    ssq_B = np.einsum('nq,nq->q', B, B, optimize=True)
    norm = np.sqrt(np.outer(ssq_B, ssq_A))  
    return c / norm

In [86]:
two_mat = two_mat_corrcoef(X_over.iloc[:,:1000], X_over.iloc[:,1000:1500])
two_mat_ein = two_mat_corrcoef_ein(X_over.iloc[:,:1000], X_over.iloc[:,1000:1500])
two_mat_np = np.corrcoef(X_over.iloc[:,:1500], rowvar=False)[1000:1500,:1000]

np.allclose(two_mat_np, two_mat, atol=0.000001) and np.allclose(two_mat_np, two_mat_ein, atol=0.000001)

True

#### two_mat_corrcoef()

In [89]:
%timeit two_mat_corrcoef(X_over.iloc[:,:10000], X_over.iloc[:,10000:20000])

1.19 s ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### two_mat_corrcoef_ein()

In [90]:
%timeit two_mat_corrcoef_ein(X_over.iloc[:,:10000], X_over.iloc[:,10000:20000], float32=True)

1.47 s ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### np.corrcoef() (for-loop)

In [91]:
%%timeit
for i in range(1, 20000):
    np.corrcoef(X_over.iloc[:,0], X_over.iloc[:,i], rowvar=False)

7.24 s ± 311 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### np.corrcoef()

In [92]:
%timeit np.corrcoef(X_over.iloc[:,:10000], X_over.iloc[:,10000:20000], rowvar=False)

7.7 s ± 263 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Chunking

In [95]:
def corr_filter_basic(X, threshold=0.9):
    corr = np.abs(corrcoef(X))
    mask = np.all(np.triu(corr, k=1) < threshold, axis=0)
    return mask

def corr_filter_chunk(X, chunk_size=20000, col_mask=None, threshold=0.9, verbose=False):
    if col_mask is None:
        col_mask = np.full((X.shape[1],), True, dtype=bool)
        
    for i in range(0, X.shape[1], chunk_size):
        if verbose:
            print(f"Processing features {i}-{min(i+chunk_size-1, X.shape[1]-1)}.")
        col_mask[i:i+chunk_size] = corr_filter(X.iloc[:, i:i+chunk_size], threshold=threshold)
    return col_mask

In [96]:
mask_9 = corr_filter_chunk(X_over, verbose=True)

Processing features 0-19999.
Processing features 20000-39999.
Processing features 40000-59999.
Processing features 60000-79999.
Processing features 80000-99999.
Processing features 100000-119999.
Processing features 120000-139999.
Processing features 140000-159999.
Processing features 160000-179999.
Processing features 180000-199999.
Processing features 200000-219999.
Processing features 220000-239999.
Processing features 240000-259999.
Processing features 260000-279999.
Processing features 280000-299999.
Processing features 300000-319999.
Processing features 320000-339999.
Processing features 340000-359999.
Processing features 360000-379999.
Processing features 380000-399999.
Processing features 400000-419999.
Processing features 420000-439999.
Processing features 440000-459999.
Processing features 460000-479999.
Processing features 480000-482739.


In [98]:
%timeit corr_filter_chunk(X_over.iloc[:,:60000], chunk_size=10000)

8.58 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### old

In [19]:
# col_mask = [True] * X_over.shape[1]
# for i in range(X_over.shape[1]-1):
#     if i % 1000 == 0:
#         print(f'Iteration complete:\t{i}')
#     if not col_mask[i]:
#         continue
#     for j in range(i+1, X_over.shape[1]):
#         if not col_mask[j]:
#             continue
#         corr = abs(np.corrcoef(X_over.iloc[:,i], X_over.iloc[:,j], rowvar=False)[0,1])
#         if corr > 0.9:
#             col_mask[j] = False
#         if (j-i) % 10000 == 0:
#             print(f'Sub-iteration complete:\t{j-i}')

In [60]:
def corr_filter(X, *, threshold, stop):
    corr = np.abs(corrcoef(X))
    avg_corr = np.average(corr, axis=0).tolist()
    if stop==0:
        return
    indices = np.argwhere(np.triu(corr, k=1) > threshold)
    if len(indices) == 0:
        return [True] * X.shape[1]
    if stop==1:
        return
    #vec = np.vectorize(avg_corr.__getitem__)(indices)
    u, inv = np.unique(indices, return_inverse=True)
    vec = np.array([avg_corr[x] for x in u])[inv].reshape(indices.shape)
    if stop == 2:
        return
    ind = np.argmax(vec, axis=1)
    if stop==3:
        return ind
    maybe_drop = indices[np.arange(indices.shape[0]), ind]
    return maybe_drop



In [47]:
%timeit corr_filter(X[:,:10000], threshold=0.5, stop=0)

1.17 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%timeit corr_filter(X[:,:10000], threshold=0.5, stop=1)

2.42 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%timeit corr_filter(X[:,:10000], threshold=0.5, stop=2)

8.51 s ± 36.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit corr_filter(X[:,:10000], threshold=0.5, stop=3)

9.5 s ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit corr_filter(X[:,:10000], threshold=0.5, stop=4)

9.85 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# np.corrcoef (v 1.20)

In [13]:
def cov(m, y=None, rowvar=True, bias=False, ddof=None, fweights=None,
        aweights=None, *, dtype=None):
    # Check inputs
    if ddof is not None and ddof != int(ddof):
        raise ValueError(
            "ddof must be integer")

    # Handles complex arrays too
    m = np.asarray(m)
    if m.ndim > 2:
        raise ValueError("m has more than 2 dimensions")

    if y is not None:
        y = np.asarray(y)
        if y.ndim > 2:
            raise ValueError("y has more than 2 dimensions")

    if dtype is None:
        if y is None:
            dtype = np.result_type(m, np.float64)
        else:
            dtype = np.result_type(m, y, np.float64)

    X = np.array(m, ndmin=2, dtype=dtype)
    if not rowvar and X.shape[0] != 1:
        X = X.T
    if X.shape[0] == 0:
        return np.array([]).reshape(0, 0)
    if y is not None:
        y = np.array(y, copy=False, ndmin=2, dtype=dtype)
        if not rowvar and y.shape[0] != 1:
            y = y.T
        X = np.concatenate((X, y), axis=0)

    if ddof is None:
        if bias == 0:
            ddof = 1
        else:
            ddof = 0

    # Get the product of frequencies and weights
    w = None
    if fweights is not None:
        fweights = np.asarray(fweights, dtype=float)
        if not np.all(fweights == np.around(fweights)):
            raise TypeError(
                "fweights must be integer")
        if fweights.ndim > 1:
            raise RuntimeError(
                "cannot handle multidimensional fweights")
        if fweights.shape[0] != X.shape[1]:
            raise RuntimeError(
                "incompatible numbers of samples and fweights")
        if any(fweights < 0):
            raise ValueError(
                "fweights cannot be negative")
        w = fweights
    if aweights is not None:
        aweights = np.asarray(aweights, dtype=float)
        if aweights.ndim > 1:
            raise RuntimeError(
                "cannot handle multidimensional aweights")
        if aweights.shape[0] != X.shape[1]:
            raise RuntimeError(
                "incompatible numbers of samples and aweights")
        if any(aweights < 0):
            raise ValueError(
                "aweights cannot be negative")
        if w is None:
            w = aweights
        else:
            w *= aweights

    avg, w_sum = np.average(X, axis=1, weights=w, returned=True)
    w_sum = w_sum[0]

    # Determine the normalization
    if w is None:
        fact = X.shape[1] - ddof
    elif ddof == 0:
        fact = w_sum
    elif aweights is None:
        fact = w_sum - ddof
    else:
        fact = w_sum - ddof*sum(w*aweights)/w_sum

    if fact <= 0:
        warnings.warn("Degrees of freedom <= 0 for slice",
                      RuntimeWarning, stacklevel=3)
        fact = 0.0

    X -= avg[:, None]
    if w is None:
        X_T = X.T
    else:
        X_T = (X*w).T
    c = np.dot(X, X_T.conj())
    c *= np.true_divide(1, fact)
    return c.squeeze()

def corrcoef(x, y=None, rowvar=True, bias=np._NoValue, ddof=np._NoValue, *,
             dtype=None):
    if bias is not np._NoValue or ddof is not np._NoValue:
        # 2015-03-15, 1.10
        warnings.warn('bias and ddof have no effect and are deprecated',
                      DeprecationWarning, stacklevel=3)
    c = cov(x, y, rowvar, dtype=dtype)
    try:
        d = np.diag(c)
    except ValueError:
        # scalar covariance
        # nan if incorrect value (nan, inf, 0), 1 otherwise
        return c / c
    stddev = np.sqrt(d.real)
    c /= stddev[:, None]
    c /= stddev[None, :]

    # Clip real and imaginary parts to [-1, 1].  This does not guarantee
    # abs(a[i,j]) <= 1 for complex arrays, but is the best we can do without
    # excessive work.
    np.clip(c.real, -1, 1, out=c.real)
    if np.iscomplexobj(c):
        np.clip(c.imag, -1, 1, out=c.imag)

    return c

In [15]:
%timeit corrcoef(X[:,:10000], rowvar=False, dtype=np.float32)

968 ms ± 8.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
